# **FAKE NEWS DETECTION**

In [2]:
import pandas as pd

In [3]:
true_df = pd.read_csv("/content/drive/MyDrive/Dataset/Project Dataset/3_Fake_News_Classifier/true.csv")

In [4]:
fake_df = pd.read_csv("/content/drive/MyDrive/Dataset/Project Dataset/3_Fake_News_Classifier/fake.csv")

In [5]:
# fake_df.isna().sum()

In [6]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
true_df["title"][3]

'FBI Russia probe helped by Australian diplomat tip-off: NYT'

In [8]:
fake_df["title"][3]

' Trump Is So Obsessed He Even Has Obama’s Name Coded Into His Website (IMAGES)'

In [9]:
max_words = true_df['title'].apply(lambda x: len(str(x).split())).max()
print("Maximum sentence length (words):", max_words)

Maximum sentence length (words): 20


In [10]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [11]:
max_words = fake_df['title'].apply(lambda x: len(str(x).split())).max()
print("Maximum sentence length (words):", max_words)

Maximum sentence length (words): 42


In [12]:
true_df["labels"]=1

In [13]:
fake_df["labels"]=0

In [14]:
df=pd.concat([true_df,fake_df]).reset_index(drop=True)

In [15]:
df.head()

,title,text,subject,date,labels
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [16]:
x=df["title"]
y=df["labels"]

In [17]:
#train and test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [18]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(35918,)
(8980,)
(35918,)
(8980,)


In [19]:
import tensorflow

In [20]:
from tensorflow.keras.layers import Embedding,Dense,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [21]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [22]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
messages = x_train.copy()
m=x_test.copy()

In [24]:
messages.head()

,title
36335,HILLARY LIES: Remember When Hillary Disclosed ...
12384,EU leaders likely to give go-ahead to new phas...
24419,Mark Hamill Is Back Mocking Trump By Reading ...
24740,Trump Gets Hilariously OWNED For Whining Abou...
27039,Obama’s Labor Secretary Zaps Trump’s Foreign ...


In [25]:
# messages.reset_index(inplace=True)
# m.reset_index(inplace=True)

In [26]:
# messages.columns

In [27]:
messages.head()

,title
36335,HILLARY LIES: Remember When Hillary Disclosed ...
12384,EU leaders likely to give go-ahead to new phas...
24419,Mark Hamill Is Back Mocking Trump By Reading ...
24740,Trump Gets Hilariously OWNED For Whining Abou...
27039,Obama’s Labor Secretary Zaps Trump’s Foreign ...


In [28]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text_series):
    corpus = []
    for i in range(len(text_series)):
        review = re.sub('[^a-zA-Z]', ' ', text_series.iloc[i])   # remove non-letters
        review = review.lower()                       # lowercase
        review = review.split()                       # tokenize
        review = [ps.stem(word) for word in review if word not in stop_words]
        review = ' '.join(review)                     # back to sentence
        corpus.append(review)
    return corpus

In [29]:
# Preprocess train and test texts separately
train_corpus = preprocess_text(messages)
test_corpus  = preprocess_text(m)

In [30]:
#tokenize
vocab_size = 5000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(messages)

In [31]:
x_train_sequence= tokenizer.texts_to_sequences(train_corpus)
x_test_sequence = tokenizer.texts_to_sequences(test_corpus)

In [32]:
# max_len = max([len(x) for x in x_train_sequence])
# print("max_len:",max_len)
max_len = 50
x_train_pad = pad_sequences(x_train_sequence, maxlen=max_len, padding='pre')
x_test_pad = pad_sequences(x_test_sequence, maxlen=max_len, padding='pre')

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# # Preprocess train and test texts separately
# train_corpus = preprocess_text(messages)
# test_corpus  = preprocess_text(m)

In [35]:
# One-hot encoding (same vocab size as training)
# vocab_size = 5000
# train_onehot = [one_hot(words, vocab_size) for words in train_corpus]
# test_onehot  = [one_hot(words, vocab_size) for words in test_corpus]

In [36]:
# # Padding (same sent_length)
# sent_length = 50
# x_train = pad_sequences(train_onehot, padding="pre", maxlen=sent_length)
# x_test  = pad_sequences(test_onehot, padding="pre", maxlen=sent_length)

In [37]:
# ### Dataset Preprocessing
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# corpus = []
# for i in range(0, len(messages)):
#     review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
#     review = review.lower()
#     review = review.split()

#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

# **One Hot Representation**

In [38]:
# vocab_size=5000
# onehot_repr=[one_hot(words,vocab_size)for words in corpus]
# onehot_repr

In [39]:
# sent_length=50
# padding_sequences = pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)

# **Model Formation**

In [40]:
#LSTM layers #Model Formation
models = Sequential()
models.add(Embedding(vocab_size,40))
models.add(LSTM(100))
models.add(Dense(1,activation="sigmoid"))
models.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [41]:
print(x_train_pad.shape, x_train.dtype)
print(y_train.shape, y_train.dtype)
print(x_test_pad.shape, x_test.dtype)
print(y_test.shape, y_test.dtype)

(35918, 50) object
(35918,) int64
(8980, 50) object
(8980,) int64


In [42]:
# Model fit
# models.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,batch_size=64)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",    # watch validation loss
    patience=3,            # stop if no improvement for 3 epochs
    restore_best_weights=True
)

history = models.fit(
    x_train_pad,y_train,
    validation_data=(x_test_pad,y_test),
    epochs=50,             # set a high max
    callbacks=[early_stop]
)


Epoch 1/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8582 - loss: 0.3070 - val_accuracy: 0.9284 - val_loss: 0.1736
Epoch 2/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9398 - loss: 0.1538 - val_accuracy: 0.9310 - val_loss: 0.1690
Epoch 3/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9490 - loss: 0.1288 - val_accuracy: 0.9333 - val_loss: 0.1655
Epoch 4/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9529 - loss: 0.1147 - val_accuracy: 0.9316 - val_loss: 0.1715
Epoch 5/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9585 - loss: 0.1083 - val_accuracy: 0.9325 - val_loss: 0.1823
Epoch 6/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9626 - loss: 0.0953 - val_accuracy: 0.9331 - val_loss: 0.1849


In [43]:
models.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,505 (2.94 MB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 513,004 (1.96 MB)

# **Adding Dropout**

In [44]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# ***Performance Metrics and Accuracy***

In [45]:
# model test
y_pred = (models.predict(x_test_pad)>0.5).astype("int32")

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [46]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[4337,  313],
       [ 286, 4044]])

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9332962138084633

In [48]:

# Save as .keras file
models.save("fake_news.keras")
print("Model got saved")

Model got saved


In [49]:
# from google.colab import files

# # Download .h5 file
# files.download("fake_news.h5")

In [50]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved as tokenizer.pkl")

✅ Tokenizer saved as tokenizer.pkl
